In [ ]:
# make sure we are working in module directory
repo_root = !git rev-parse --show-toplevel
module_path = repo_root[0] + "/backend/heatflask"
%cd $module_path

In [ ]:
# %%writefile Streams.py
"""
***  For Jupyter notebook ***
Paste one of these Jupyter magic directives to the top of a cell
 and run it, to do these things:
    %%cython --annotate    # Compile and run the cell
    %load Streams.py         # Load Streams.py file into this (empty) cell
    %%writefile Streams.py   # Write the contents of this cell to Streams.py
"""

import os
import time
import datetime
from logging import getLogger
import numpy as np

import DataAPIs
from DataAPIs import redis
import Strava

log = getLogger(__name__)
log.propagate = True

APP_NAME = "heatflask"
COLLECTION_NAME = "streams"
CACHE_PREFIX = "S:"

SECS_IN_HOUR = 60 * 60
SECS_IN_DAY = 24 * SECS_IN_HOUR

MONGO_TTL = int(os.environ.get("MONGO_STREAMS_TTL", 10)) * SECS_IN_DAY
REDIS_TTL = int(os.environ.get("REDIS_STREAMS_TTL", 4)) * SECS_IN_HOUR

DATA = {}


async def get_collection():
    if "col" not in DATA:
        DATA["col"] = await DataAPIs.init_collection(
            COLLECTION_NAME, ttl=MONGO_TTL, cache_prefix=CACHE_PREFIX
        )
    return DATA["col"]


def mongo_doc(activity_id, stream_data, ts=None):
    return {
        "_id": int(activity_id),
        "mpk": stream_data,
        "ts": ts or datetime.datetime.now(),
    }

def cache_key(aid):
    return f"{CACHE_PREFIX}{aid}"


async def strava_import(activity_ids, **user):
    uid = int(user["_id"])
    
    strava = Strava.AsyncClient(uid, **user["auth"])
    await strava.update_access_token()
    
    streams = await get_collection()
    
    docs = []
    now = datetime.datetime.now()
    aiterator = strava.get_many_streams(activity_ids)
    async for item in aiterator:
        abort_signal = yield item
        
        try:
            doc = mongo_doc(*item, ts=now)
        except Exception as e:
            log.error("Streams %s encode error: %s", activity_id, e)
        else:
            docs.append(doc)
            
        if abort_signal:
            await Strava.AsyncClient.abort(aiterator)
            break
    
    # now store all this stuff
    await delete([doc["_id"] for doc in docs])
    await streams.insert_many(docs)
    
    async with redis.pipeline(transaction=True) as pipe:
        for doc in docs:
            key = cache_key(doc["_id"]) 
            await pipe.setex(key, REDIS_TTL, doc['mpk'])
        redis_result = await pipe.execute()
        

async def aiter_query(activity_ids=None, limit=0, update_ts=False, user=None):
    if not activity_ids:
        return
    
    # First we check Redis cache
    keys = [cache_key(aid) for aid in activity_ids]
    redis_query_results = await redis.mget(self, keys)
    log.info(redis_query_results)
    
    activity_ids2 = set()
    async with redis.pipeline(transaction=True) as pipe:
        for i, s in enumerate(redis_query_results):
            aid = activity_ids[i]
            if s:
                yield aid, s
                pipe.expire(keys[i], REDIS_TTL)
            else:
                activity_ids2.add(aid)
    
#         log.debug("retrieved %d from redis in %d")
        if not len(activities2):
            pipe.execute()
            return
        
        # Then we check MongoDB
        streams = await get_collection()
        query = {"_id": {"$in": list(activity_ids2)}}
    
        t0 = time.perf_counter()
        cursor = streams.find(query, limit=limit)
        from_mongo = set()
        async for doc in cursor:
            aid = doc["_id"]
            streams_data = doc["mpk"]
            yield aid, streams_data
            from_mongo.add(aid)
            await pipe.setex(cache_key(aid), REDIS_TTL, streams_data)
        await pipe.execute()
    
    n = len(ids)
    if update_ts and n:
        t0 = time.perf_counter()
        update_result = await streams.update_many(
            {"_id": {"$in": ids}},
            {"$set": {"ts": datetime.datetime.utcnow()}},
        )
        elapsed = (time.perf_counter() - t0) * 1000
        log.debug("streams update ts: %d records in %dms", n, elapsed)

    # Finally, if there are any activities that we don't have streams for, 
    #  import them from Strava.
    remaining_ids = activities2 - from_mongo
    if remaining_ids and user is not None:
        async for aid, s in strava_import(list(remaining_ids), **user):
            yield aid, s

async def query(**kwargs):
    return [s async for s in aiter_query(**kwargs)]
        
async def delete(activity_ids):
    streams = await get_collection()
    return await streams.delete_many({"_id": {"$in": activity_ids}})



In [ ]:
import logging
logging.basicConfig(level="DEBUG")

N_FETCH = 10

import Index
result = await Index.query(limit=10)
activity_ids = [d["_id"] for d in result["docs"]]
result

In [ ]:
import asyncio
import Users
import Strava

admin = await Users.get(Users.ADMIN[0])
admin

In [ ]:
gen = strava_import(activity_ids, **admin)
s1 = await gen.__anext__()
# await asyncio.sleep(0.5)
s2 = await gen.__anext__()
# await asyncio.sleep(0.5)
await Strava.AsyncClient.abort(gen)

In [ ]:
await query(activity_ids=activity_ids, update_ts=True)

In [ ]:
keys = await redis.keys("*")
await redis.get(keys[0])